In [1]:
import requests
from requests import Response
from pydantic import RootModel,BaseModel,Field,field_validator
from datetime import datetime
import pandas as pd 

class Site (BaseModel):
    行政區域:str = Field(alias='sarea')
    總數量:int = Field(alias='total')
    可借車量:int = Field(alias='available_rent_bikes') 
    可還車量:int = Field(alias='available_return_bikes')
    時間:datetime = Field(alias='mday')

    @field_validator('可借車量','可還車量',mode='before')
    @classmethod
    def whitespace_to_zero(cls, value: str) -> str:
        return '0.0' if value == '' else value
    
class Youbike(RootModel):
    root:list[Site]

try:
    youbike_url:Response= requests.get('https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json')
    youbike_url.raise_for_status()
except Exception as e:
    print(e)
else:
    data = Youbike.model_validate_json(youbike_url.text)
    all_sites = data.model_dump()

df = pd.DataFrame(all_sites)
df

,行政區域,總數量,可借車量,可還車量,時間
0,大安區,28,0,28,2024-07-01 20:29:16
1,大安區,21,0,21,2024-07-01 20:21:14
2,大安區,16,6,10,2024-07-01 20:26:19
3,大安區,11,0,11,2024-07-01 20:19:14
4,大安區,16,3,13,2024-07-01 20:27:19
...,...,...,...,...,...
1424,臺大公館校區,30,1,28,2024-07-01 20:28:15
1425,臺大公館校區,20,2,18,2024-07-01 20:27:19
1426,臺大公館校區,24,2,21,2024-07-01 20:25:23
1427,臺大公館校區,40,30,1,2024-07-01 20:27:19


Lesson21_0

In [2]:
df1 = df.set_index(keys=["行政區域"])
df1.groupby(level='行政區域')[['總數量','可借車量','可還車量']].sum()
#groupby實體.若是一個統計的(function)、若是多個

,總數量,可借車量,可還車量
行政區域,,,
中山區,3819,1609,2176
中正區,3209,1521,1661
信義區,3331,1245,1980
內湖區,3499,1481,1978
北投區,2472,909,1532
南港區,2317,1103,1194
士林區,2937,1024,1829
大同區,1629,602,1013
大安區,4987,2103,2780


In [3]:
df1 = df.set_index(keys=['行政區域'])
df1.groupby(level='行政區域')[['總數量','可借車量','可還車量']].sum()

,總數量,可借車量,可還車量
行政區域,,,
中山區,3819,1609,2176
中正區,3209,1521,1661
信義區,3331,1245,1980
內湖區,3499,1481,1978
北投區,2472,909,1532
南港區,2317,1103,1194
士林區,2937,1024,1829
大同區,1629,602,1013
大安區,4987,2103,2780


In [4]:
#pivot
#stack,unstack
#欄位有名稱
df1.stack()

行政區域        
大安區     總數量                      28
        可借車量                      0
        可還車量                     28
        時間      2024-07-01 20:29:16
        總數量                      21
                       ...         
臺大公館校區  時間      2024-07-01 20:27:19
        總數量                      18
        可借車量                      2
        可還車量                     16
        時間      2024-07-01 20:28:20
Length: 5716, dtype: object